In [1]:
%load_ext autoreload
%autoreload 2

In [14]:
import numpy as np
import pandas as pd
import ssm
import matplotlib.pyplot as plt
import plotly.express as px
import pickle
import scipy
import scipy.io
import tsfresh
import dtale
import os
px.set_mapbox_access_token("pk.eyJ1IjoibWlrb2xhanNsdXBpbnNraSIsImEiOiJjazJ5dDI0MWEwOTA3M2hxanRwbmo4NDRuIn0.NSbj3B1Bm6fC5QUP3s-BjQ")
import plotly.io as pio
pio.renderers.default = "iframe"
import dask
from lib.huawei import load_data, load_motion, load_ambient, load_battery, load_api, load_label
import tables
from ssm.util import random_rotation, find_permutation
import logging
from sklearn.metrics import accuracy_score, confusion_matrix
import threading
logging.basicConfig(filename='huawei_reduced_hmm_parallel.log', level=logging.DEBUG)
from multiprocessing import Pool

In [2]:
with open("../Data/huawei-competition/selected_features.pickle", "rb") as ifile:
    selected_features = pickle.load(ifile)

In [3]:
data = []
for selected_feature in selected_features[:20]:
#     file_path = os.path.join("../Data/2012.01.17-huawei/splitted/", f"1.3a-stationary_50ms__{selected_feature}.csv")
#     df = pd.read_csv(file_path, parse_dates = ["DATETIME_UTC"], index_col="DATETIME_UTC")
    a = np.load(os.path.join("../", selected_feature))
    data.append(a)
#     print(df.shape)
data = np.stack(data, axis=1)

In [4]:
labels = np.load("../Data/huawei-competition/resampled/Label.npy")[:,-1].astype(int) - 1

In [5]:
order = np.loadtxt("../Data/huawei-competition/train_order.txt").astype(int)

In [6]:
data_ordered = np.zeros(data.shape)

In [7]:
data_ordered[order - 1, :] = data

In [8]:
labels_ordered = np.zeros(labels.shape)
labels_ordered[order - 1] = labels

In [9]:
n = int(data_ordered.shape[0]*0.25)
m = int(data_ordered.shape[0]*0.5)
X_train = data_ordered[m:]
y_train = labels_ordered[m:]
X_val = data_ordered[:n]
y_val = labels_ordered[:n]
X_test = data_ordered[n:m]
y_test = labels_ordered[n:m]

In [10]:
def train_model(X_train,
                y_train,
                transitions="recurrent",
                dynamics="diagonal_gaussian",
                single_subspace=False,
                num_iters=10000,
                num_init_iters=10000,
                num_init_restarts=3,
                discrete_state_init_method = "kmeans"
               ):
    arhmm = ssm.hmm.HMM(8, 20,
                            transitions=transitions,
                            observations=dynamics)
    hmm_lls = arhmm.fit(X_train, num_iters=num_iters,
             num_init_iters=num_init_iters, init_method = discrete_state_init_method)
    most_likely = arhmm.most_likely_states(X_train)
    try:
        perm = find_permutation(y_train.astype(int), most_likely)
        arhmm.permute(perm)
    except:
        logging.error("Permutation exception")
    with open(f"../Models/hmm_parallel/huawei_reduced_hmm_{transitions}_{dynamics}_{num_iters}_{num_init_iters}_{discrete_state_init_method}.pickle", "wb") as ofile:
        pickle.dump({
            "arhmm": arhmm,
            "hmm_lls": hmm_lls,
            "most_likely": most_likely,
        }, ofile)
    return {
            "arhmm": arhmm,
            "hmm_lls": hmm_lls,
            "most_likely": most_likely,
        }

In [11]:
params_list = [
#     {
#         "transitions": "recurrent",
#         "dynamics": "diagonal_gaussian",
#         "num_iters": 10,
#         "discrete_state_init_method": "random"
#     },
#     {
#         "transitions": "recurrent",
#         "dynamics": "diagonal_gaussian",
#         "num_iters": 1000,
#         "discrete_state_init_method": "random"
#     },
#     {
#         "transitions": "recurrent",
#         "dynamics": "diagonal_gaussian",
#         "num_iters": 1000,
#         "discrete_state_init_method": "kmeans"
#     },
#     {
#         "transitions": "recurrent",
#         "dynamics": "diagonal_gaussian",
#         "num_iters": 10000,
#         "discrete_state_init_method": "random"
#     },
#     {
#         "transitions": "recurrent",
#         "dynamics": "diagonal_gaussian",
#         "num_iters": 10000,
#         "discrete_state_init_method": "kmeans"
#     },
    {
        "transitions": "recurrent",
        "dynamics": "diagonal_gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent",
        "dynamics": "diagonal_gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent",
        "dynamics": "autoregressive",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent",
        "dynamics": "autoregressive",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent",
        "dynamics": "no_input_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent",
        "dynamics": "no_input_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent",
        "dynamics": "diagonal_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent",
        "dynamics": "diagonal_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent",
        "dynamics": "robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent",
        "dynamics": "robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent",
        "dynamics": "diagonal_robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent",
        "dynamics": "diagonal_robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent",
        "dynamics": "gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent",
        "dynamics": "gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "standard",
        "dynamics": "diagonal_gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "standard",
        "dynamics": "diagonal_gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "standard",
        "dynamics": "autoregressive",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "standard",
        "dynamics": "autoregressive",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "standard",
        "dynamics": "no_input_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "standard",
        "dynamics": "no_input_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "standard",
        "dynamics": "diagonal_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "standard",
        "dynamics": "diagonal_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "standard",
        "dynamics": "robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "standard",
        "dynamics": "robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "standard",
        "dynamics": "diagonal_robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "standard",
        "dynamics": "diagonal_robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "standard",
        "dynamics": "gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "standard",
        "dynamics": "gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "diagonal_gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "autoregressive",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "autoregressive",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "no_input_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "no_input_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "diagonal_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "diagonal_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "diagonal_robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "diagonal_robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "recurrent_only",
        "dynamics": "gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "standard",
        "dynamics": "diagonal_gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "standard",
        "dynamics": "diagonal_gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "standard",
        "dynamics": "autoregressive",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "stationary",
        "dynamics": "autoregressive",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "stationary",
        "dynamics": "no_input_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "stationary",
        "dynamics": "no_input_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "stationary",
        "dynamics": "diagonal_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "stationary",
        "dynamics": "diagonal_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "stationary",
        "dynamics": "robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "stationary",
        "dynamics": "robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "stationary",
        "dynamics": "diagonal_robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "stationary",
        "dynamics": "diagonal_robust_ar",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
    {
        "transitions": "stationary",
        "dynamics": "gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "random"
    },
    {
        "transitions": "stationary",
        "dynamics": "gaussian",
        "num_iters": 10000,
        "discrete_state_init_method": "kmeans"
    },
]

In [12]:
def train_successfully(params):
    try:
        train_model(X_train, y_train, **params)
    except Exception as e:
        logging.error(f"{e}")

In [ ]:
with Pool(processes=4) as pool:
    pool.map(train_successfully, params_list)

/pio/scratch/1/i280025/miniconda3/lib/python3.7/site-packages/ssm/regression.py:495: UserWarning:

generalized_newton_studentst_dof fixed point grew beyond bounds [1e-08,100] to 5.4935436200054255e-09.

/pio/scratch/1/i280025/miniconda3/lib/python3.7/site-packages/ssm/regression.py:495: UserWarning:

generalized_newton_studentst_dof fixed point grew beyond bounds [1e-08,100] to 2.8694677745022545e-09.

/pio/scratch/1/i280025/miniconda3/lib/python3.7/site-packages/ssm/regression.py:495: UserWarning:

generalized_newton_studentst_dof fixed point grew beyond bounds [1e-08,100] to 2.8817039865458582e-09.

/pio/scratch/1/i280025/miniconda3/lib/python3.7/site-packages/ssm/regression.py:495: UserWarning:

generalized_newton_studentst_dof fixed point grew beyond bounds [1e-08,100] to 2.829686263083886e-09.

/pio/scratch/1/i280025/miniconda3/lib/python3.7/site-packages/ssm/regression.py:495: UserWarning:

generalized_newton_studentst_dof fixed point grew beyond bounds [1e-08,100] to 2.771193274

In [ ]:
def plot_states(data_z, z_est, label, gen):
    titles = ["True", label]
    states_list = [data_z, z_est]
    fig, axs = plt.subplots(2,1, figsize=(6,4))
    for (i, ax, states) in zip(range(len(axs)), axs, states_list):
        ax.imshow(states[None,:], aspect="auto")
        ax.set_yticks([])
        ax.set_title(titles[i])
        if i < (len(axs) - 1):
            ax.set_xticks([])

    plt.suptitle(f"{titles[0]} and {titles[1]} States: {gen}", va="baseline")
    plt.tight_layout()
    plt.savefig(f"../Plots/hmm/{prefix}_{titles[0]}_{titles[1]}.pdf")
    plt.show()

In [ ]:
"test"

In [ ]:
plot_states(y_train, models[0]["most_likely"], "estimated", 0 )

In [ ]:
accuracy_score(y_train, models[1]["most_likely"])

In [ ]:
confusion_matrix(y_train, models[1]["most_likely"])

In [ ]:
posterior_x = posterior.mean_continuous_states[0]
most_likely = rslds.most_likely_states(posterior_x, X_train)

In [ ]:
y_train

In [ ]:
# try:
perm = find_permutation(y_train.astype(int), most_likely)
rslds.permute(perm)
# except:
#     logging.error("Prediction exception")

In [ ]:
z_est = rslds.most_likely_states(posterior_x, X_train)
plot_states(y_train, z_est, "Predicted", 0)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
accuracy_score(y_train, z_est)

In [ ]:
confusion_matrix(y_train, z_est)

In [ ]:
print(models[0]['hmm_lls'])

In [ ]:
plt.plot(models[1]['hmm_lls'][1000:])

In [ ]:
"test"